In [1]:
#!pip3 install -qU xgboost optuna pandas scipy scikit-learn matplotlib tqdm 

In [2]:
import random
import torch
import numpy as np


SEED = 1000

def setup_reproducibility():
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False, warn_only=True)
    torch.set_float32_matmul_precision("high")

setup_reproducibility()

In [3]:
from sklearn.model_selection import train_test_split


def get_stats(tensor, p=True, r=False):
    mean, std = tensor.mean(), tensor.std()
    min, max =  tensor.min(), tensor.max()
    
    if p: print(f"Min: {min}, Max: {max}, Mean: {mean}, Std: {std}")
    if r: return min, max, mean, std
    
    
def zscore(tensor, mean=None, std=None):
    if mean is None: mean = tensor.mean()
    if std is None: std = tensor.std()
    return (tensor - mean) / (std + 1e-6)


def get_model_size(model):
    print(sum(p.numel() for p in model.parameters()) / 1e6)
    

def get_index(iterable):
    return random.randint(0, len(iterable) - 1)


def split(inputs, targets, seed):
    return train_test_split(
        inputs,
        targets, 
        test_size=0.2,
        shuffle=True, 
        random_state=seed
    ) 

In [4]:
import matplotlib.pyplot as plt


def show_waves(waves, dpi=100):
    """
    waves: numpy array of shape (3, N)
    Creates three separate figures that stretch wide.
    """

    N = waves.shape[1]
    t = np.arange(N)

    # Wide aspect ratio; height modest so each window fills width
    for i in range(waves.shape[0]):
        fig = plt.figure(figsize=(14, 4), dpi=dpi)  # wide figure
        ax = fig.add_subplot(111)
        ax.plot(t, waves[i], linewidth=1)
        ax.set_title(f"Wave {i+1}")
        ax.set_xlabel("Sample")
        ax.set_ylabel("Amplitude")
        ax.grid(True)
        fig.tight_layout()  # reduce margins to use width
    plt.show()

In [5]:
from utils import hf_ds_download

HF_TOKEN = "xhf_XURkoNhwOIPtEdHfNeRpVkjEwKSkhtigFi"
path = hf_ds_download(HF_TOKEN, repo_id="ArbaazBeg/kaggle-spectogram")

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

sample_submission.csv:   0%|          | 0.00/611 [00:00<?, ?B/s]

96_samples.csv:   0%|          | 0.00/1.99M [00:00<?, ?B/s]

anton_785.csv:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

metrohm.csv:   0%|          | 0.00/5.39M [00:00<?, ?B/s]

...

timegate.csv:   0%|          | 0.00/808k [00:00<?, ?B/s]

transfer_plate.csv:   0%|          | 0.00/2.02M [00:00<?, ?B/s]

mettler_toledo.csv:   0%|          | 0.00/13.5M [00:00<?, ?B/s]

tec5.csv:   0%|          | 0.00/15.7M [00:00<?, ?B/s]

tornado.csv:   0%|          | 0.00/19.5M [00:00<?, ?B/s]

In [6]:
import os

#path = "/root/.cache/huggingface/hub/datasets--ArbaazBeg--kaggle-spectogram/snapshots/b61d17629d4886fcc89e5bd9ca022af4da493d73"
files = sorted(os.listdir(path))
[(i, files[i]) for i in range(len(files))]

[(0, '.gitattributes'),
 (1, '96_samples.csv'),
 (2, 'anton_532.csv'),
 (3, 'anton_785.csv'),
 (4, 'kaiser.csv'),
 (5, 'metrohm.csv'),
 (6, 'mettler_toledo.csv'),
 (7, 'sample_submission.csv'),
 (8, 'tec5.csv'),
 (9, 'timegate.csv'),
 (10, 'tornado.csv'),
 (11, 'transfer_plate.csv')]

In [7]:
import pandas as pd


csv_path = os.path.join(path, files[11])
df = pd.read_csv(csv_path)

input_cols = df.columns[1:2049]
target_cols = df.columns[2050:]

targets  = df[target_cols].dropna().to_numpy()

df = df[input_cols]
df['Unnamed: 1'] = df['Unnamed: 1'].str.replace("[\[\]]", "", regex=True).astype('int64')
df['Unnamed: 2048'] = df['Unnamed: 2048'].str.replace("[\[\]]", "", regex=True).astype('int64')

inputs = df.to_numpy().reshape(-1, 2, 2048)
inputs = inputs.mean(axis=1)

inputs.shape, targets.shape

<>:13: SyntaxWarning: invalid escape sequence '\['
<>:14: SyntaxWarning: invalid escape sequence '\['
<>:13: SyntaxWarning: invalid escape sequence '\['
<>:14: SyntaxWarning: invalid escape sequence '\['
/tmp/ipykernel_8378/3793664017.py:13: SyntaxWarning: invalid escape sequence '\['
  df['Unnamed: 1'] = df['Unnamed: 1'].str.replace("[\[\]]", "", regex=True).astype('int64')
/tmp/ipykernel_8378/3793664017.py:14: SyntaxWarning: invalid escape sequence '\['
  df['Unnamed: 2048'] = df['Unnamed: 2048'].str.replace("[\[\]]", "", regex=True).astype('int64')


((96, 2048), (96, 3))

In [8]:
import numpy as np
from scipy import signal
from tqdm.auto import tqdm


def get_advanced_spectra_features(X):
    """Create multi-channel features from spectra: raw, 1st derivative, 2nd derivative."""
    X_processed = np.zeros_like(X)
    # Baseline correction and SNV
    for i in tqdm(range(X.shape[0])):
        poly = np.polyfit(np.arange(X.shape[1]), X[i], 3)
        baseline = np.polyval(poly, np.arange(X.shape[1]))
        corrected_spec = X[i] - baseline
        X_processed[i] = (corrected_spec - corrected_spec.mean()) / (corrected_spec.std() + 1e-8)

    # Calculate derivatives
    deriv1 = signal.savgol_filter(X_processed, window_length=11, polyorder=3, deriv=1, axis=1)
    deriv2 = signal.savgol_filter(X_processed, window_length=11, polyorder=3, deriv=2, axis=1)

    # Stack as channels
    return np.stack([X_processed, deriv1, deriv2], axis=1)

inputs = get_advanced_spectra_features(inputs)
inputs.shape

  0%|          | 0/96 [00:00<?, ?it/s]

(96, 3, 2048)

In [9]:
inputs = inputs.reshape(-1, 3 * 2048).astype(np.float32)
targets = targets.astype(np.float32)

In [10]:
train_inputs, eval_inputs, train_targets, eval_targets = split(inputs, targets, SEED)

In [11]:
_, _, mean, std = get_stats(train_inputs, p=False, r=True)
train_inputs = zscore(train_inputs, mean, std)
eval_inputs = zscore(eval_inputs, mean, std)

In [ ]:
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score


def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 200, 1000)
    max_depth = trial.suggest_int("max_depth", 3, 20)
    learning_rate = trial.suggest_float("learning_rate", 0.0001, 0.8, log=True)
    
    model = XGBRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=SEED,
        n_jobs=-1,
        #reg_lambda=1.0,
        eval_metric="rmse",  
        #early_stopping_rounds=5,        
        tree_method="hist", 
        device="cuda",
    )

    model.fit(
        train_inputs, train_targets,
        eval_set=[(eval_inputs, eval_targets)],
        verbose=False,
    )

    preds = model.predict(eval_inputs)
    return r2_score(eval_targets, preds)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, n_jobs=4)

[I 2025-08-21 11:26:41,045] A new study created in memory with name: no-name-76e2879f-cbb6-4c75-b800-b99c55c6b290
[I 2025-08-21 11:27:04,429] Trial 2 finished with value: -0.1607435941696167 and parameters: {'n_estimators': 390, 'max_depth': 17, 'learning_rate': 0.0025102287166713613}. Best is trial 2 with value: -0.1607435941696167.
[I 2025-08-21 11:27:07,409] Trial 3 finished with value: 0.8693812489509583 and parameters: {'n_estimators': 593, 'max_depth': 17, 'learning_rate': 0.10780321329415883}. Best is trial 3 with value: 0.8693812489509583.
[I 2025-08-21 11:27:10,230] Trial 1 finished with value: 0.6935000419616699 and parameters: {'n_estimators': 477, 'max_depth': 8, 'learning_rate': 0.003635902388497257}. Best is trial 3 with value: 0.8693812489509583.
[I 2025-08-21 11:27:33,418] Trial 4 finished with value: 0.9041867256164551 and parameters: {'n_estimators': 434, 'max_depth': 17, 'learning_rate': 0.006955357875612961}. Best is trial 4 with value: 0.9041867256164551.
[I 2025-0

In [14]:
import optuna 

study = optuna.load_study(study_name="xgb", storage="sqlite:///experiements.db")

In [15]:
study.best_params, study.best_value

({'n_estimators': 543,
  'learning_rate': 0.008173583900478528,
  'max_depth': 26,
  'reg_lambda': 0.03218976847790143,
  'early_stopping_rounds': 134},
 0.9259349703788757)

In [17]:
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

model = XGBRegressor(
    n_estimators=study.best_params["n_estimators"],
    learning_rate=study.best_params["learning_rate"],
    max_depth=study.best_params["max_depth"],
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=SEED,
    n_jobs=-1,
    reg_lambda=study.best_params["reg_lambda"],
    eval_metric="rmse",  
    early_stopping_rounds=study.best_params["early_stopping_rounds"],      
    tree_method="hist", 
    device="cuda",
)

model.fit(
    train_inputs, train_targets,
    eval_set=[(eval_inputs, eval_targets)],
    verbose=False,
)

preds = model.predict(eval_inputs)
r2_score(eval_targets, preds)

/venv/main/lib/python3.12/site-packages/xgboost/core.py:729: UserWarning: [05:31:29] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


0.9259349703788757

In [18]:
r2_score(eval_targets, preds)

0.9143840670585632

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from tqdm.auto import tqdm


seeds = [SEED]#np.random.randint(0, 10**6, size=10).tolist()
models = {"1": [], "2": [], "3": []}

for seed in tqdm(seeds):
    print(f"Seed {seed}")
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)
    scores_mean = []
    
    for i in range(3):
        splits = kf.split(inputs, targets)
        scores = []
        
        for j, (train_idx, eval_idx) in enumerate(splits):
            train_inputs, train_targets = inputs[train_idx], targets[train_idx, i]
            eval_inputs, eval_targets = inputs[eval_idx], targets[eval_idx, i]
            
            _, _, mean, std = get_stats(train_inputs, p=False, r=True)
            train_inputs = zscore(train_inputs, mean, std)
            eval_inputs = zscore(eval_inputs, mean, std)
            
            model = XGBRegressor(
                n_estimators=200,
                learning_rate=0.05,
                max_depth=3,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=seed,
                n_jobs=-1,
                #reg_lambda=1.0,
                eval_metric="rmse",  
                #early_stopping_rounds=5,        
                tree_method="hist", 
                device="cuda",
            )
            
            model.fit(
                train_inputs, train_targets,
                eval_set=[(eval_inputs, eval_targets)],
                verbose=False,
            )
            
            preds = model.predict(eval_inputs)
            score = r2_score(eval_targets, preds)
            scores.append(score)
            models[str(i+1)].append((score, model, mean, std))
        
        scores = np.mean(scores)
        scores_mean.append(scores)
        print(f"Mean R2 (target {i}): {scores:.4f}")
    
    scores_mean = np.mean(scores_mean)
    print(f"Final: {scores_mean:.4f}\n")

In [ ]:
def get_best_models(models):
    best_models = []
    
    for i in range(1, 4):
        score = float('-inf')
        for s, model, mean, std in models[str(i)]:
            if s > score:
                score = s
                best = model
                mu = mean
                sigma = std
        best_models.append((best, score, mu, sigma))
            
    return best_models

best_models = get_best_models(models)

In [ ]:
scores = {"1": [], "2": [], "3": []}
for i in range(1, 4):
    for s, m, mean, std in models[str(i)]:
        scores[str(i)].append(s)    
    scores[str(i)] = np.mean(scores[str(i)])
    
scores, np.mean([scores["1"], scores["2"], scores["3"]])

In [ ]:
scores = []
for m, s, mu, sigma in best_models:
    scores.append(s)
scores, np.mean(scores)

In [18]:
csv_path = os.path.join(path, files[1])
test_df = pd.read_csv(csv_path)


row1 = test_df.columns[1:].to_numpy().copy()
row1[-1] = "5611"
row1 = row1.astype(np.float64)


cols = test_df.columns[1:]
test_df = test_df[cols]
test_df[" 5611]"] = test_df[" 5611]"].str.replace('[\[\]]', '', regex=True).astype('int64')
test = test_df.to_numpy()

test = np.insert(test, 0, row1, axis=0)
test = test.reshape(-1, 2, 2048).mean(axis=1)

get_stats(test)
test = get_advanced_spectra_features(test)
test = test.reshape(-1, 3 * 2048)
test = zscore(test, mean, std)
test.shape, test.dtype, get_stats(test)

<>:12: SyntaxWarning: invalid escape sequence '\['
<>:12: SyntaxWarning: invalid escape sequence '\['
/tmp/ipykernel_8378/2309359108.py:12: SyntaxWarning: invalid escape sequence '\['
  test_df[" 5611]"] = test_df[" 5611]"].str.replace('[\[\]]', '', regex=True).astype('int64')


Min: 983.5, Max: 65535.0, Mean: 3623.216623942057, Std: 6772.114021862654


  0%|          | 0/96 [00:00<?, ?it/s]

Min: -7.431345851215631, Max: 23.697375375134172, Mean: -0.00016526900992435334, Std: 1.0056484999072834


((96, 6144), dtype('float64'), None)

In [19]:
preds = model.predict(test)
preds.shape

(96, 3)

In [ ]:
preds = []

for i in range(3):
    local_preds = []
    for _, model, mean, std in models[str(i + 1)]:
        t = zscore(test.copy(), mean, std)
        p = model.predict(test)
        local_preds.append(p)
    
    local_preds = np.stack(local_preds).mean(axis=0)
    preds.append(local_preds)
    
preds = np.column_stack(preds)
preds.shape

In [ ]:
preds = []

for model, s, mean, std in best_models:
    print(mean, std)
    t = zscore(test.copy(), mean, std)
    p = model.predict(t)
    preds.append(p)
    
preds = np.column_stack(preds)
preds.shape

In [ ]:
preds = model.o

In [23]:
column_names = ['Glucose', 'Sodium Acetate', 'Magnesium Sulfate']
preds_df = pd.DataFrame(preds, columns=column_names)
preds_df.insert(0, 'ID', [i+1 for i in range(len(preds_df))])
preds_df

,ID,Glucose,Sodium Acetate,Magnesium Sulfate
0,1,3.206942,0.909356,0.590890
1,2,6.820968,1.962672,1.805737
2,3,4.210983,0.474982,1.159761
3,4,3.949406,0.960254,0.582332
4,5,9.441555,0.991336,1.285040
...,...,...,...,...
91,92,4.732452,0.798559,1.031371
92,93,4.856393,0.471741,1.102562
93,94,3.909436,0.482411,0.913660
94,95,3.492605,1.391877,1.117882


In [27]:
name = "/xgboost.optuna.9143.csv"
preds_df.to_csv(name, index=False)
f = pd.read_csv(f"{name}")
f

,ID,Glucose,Sodium Acetate,Magnesium Sulfate
0,1,3.206942,0.909355,0.590890
1,2,6.820968,1.962671,1.805737
2,3,4.210983,0.474982,1.159761
3,4,3.949406,0.960254,0.582332
4,5,9.441555,0.991336,1.285040
...,...,...,...,...
91,92,4.732452,0.798559,1.031371
92,93,4.856393,0.471741,1.102562
93,94,3.909436,0.482411,0.913660
94,95,3.492605,1.391877,1.117882
